<a href="https://colab.research.google.com/github/kevinhhl/Weather-to-Fly-WtF-/blob/main/Weather_to_Fly_(Hong_Kong)_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Configurations**
This is where all configs will be set before the script makes an API call [via openmeteo] to collect weather data.

In [1]:
locations = {\
             "22.28185990473875, 114.18903700821926" : "Victoria Park",\
             "22.27054210988102, 114.00032030347596"  : "Mui Wo", \
             "22.540797536909047, 114.43437970480949" : "Tung Ping Chau", \
             "22.29770991812577, 114.30310328185212" : "Clear Water Bay", \
             "22.239390991398583, 114.24173429924463" : "Dragon's Back",\
             "22.468108498629288, 114.36264537675595" : "Tap Mun", \
             "22.429160036257116, 114.23296055659219" : "Ma On Shan", \
             "22.23398579735171, 114.15930196830752" : "Ap Lei Chau", \
             "22.28617189441449, 114.04065619628331" : "Peng Chau"\
             }


MAX_WIND_kph     = 35    # [DJI Mini SE] wind resistance of up to 37.8 km/h
MAX_PRECIP_mm    = 0
MAX_CLOUDS_pcnt  = 50

#**Processing of raw data**

In [2]:
import collections
locations = collections.OrderedDict(sorted(locations.items()))

In [3]:
from urllib.request import urlopen, Request
import json
import pandas as pd
import datetime
import pytz

def get_flytimes(latlong, name, maxwind_kph, maxprecip_mm, maxclouds_pcnt, is_verbose):
  # Construct URL
  gps_loc = latlong.split(",")
  latitude = str(round(float(gps_loc[0].strip()),4))
  longitude = str(round(float(gps_loc[1].strip()),4))
  url = 'https://api.open-meteo.com/v1/forecast?latitude=' + latitude + '&longitude=' + longitude +'&hourly=precipitation,cloudcover,windgusts_10m&timezone=auto'
  if is_verbose:
    print(url)
    print("Weather Forecast: {} \nGPS: {}".format(name, latlong))

  # Retrieve data
  request = Request(url)
  response = urlopen(request)
  forecast = response.read()
  data = json.loads(forecast)
  df_json = pd.json_normalize(data)

  list_gust = df_json["hourly.windgusts_10m"][0]
  list_time = df_json["hourly.time"][0]
  list_precip = df_json["hourly.precipitation"][0]
  list_cloud = df_json["hourly.cloudcover"][0]
  assert len(list_gust) == len(list_time) and len(list_time) == len(list_precip) and len(list_precip) == len(list_cloud)

  # Init. custom DF
  timezone = df_json['timezone'][0]
  COL_TIME = "Datetime (" + timezone + ")"
  COL_GUST = "Gust (" + df_json['hourly_units.windgusts_10m'][0] + ")"
  COL_PRECIP = "Precip. (" + df_json['hourly_units.precipitation'][0] + ")"
  COL_CLOUD = "Cloud Cover (" + df_json['hourly_units.cloudcover'][0] + ")"

  # Constructing datetime filter
  tz = df_json["timezone"][0]
  datetimenow = datetime.datetime.now(pytz.timezone(tz))
  datefilter = []
  list_dayofweek = []
  for i in range(len(list_time)):
    substr1 = list_time[i].split('-')
    yyyy, mm = int(substr1[0]), int(substr1[1])
    substr2 = substr1[2].split('T')
    dd, hh = int(substr2[0]), int(substr2[1].split(':')[0])
    is_daytime = hh < 18 and hh > 6
    datefilter.append(dd >= datetimenow.day and mm >= datetimenow.month and yyyy >= datetimenow.year and is_daytime)
    str_dayofweek = datetime.date(yyyy,mm,dd).strftime('%A')
    list_dayofweek.append(str_dayofweek)
    list_time[i] = list_time[i]+ " " + str_dayofweek.ljust(10, ' ') 

  df_custom = pd.DataFrame({COL_TIME : list_time, \
                            COL_GUST : list_gust, \
                            COL_PRECIP : list_precip, \
                            COL_CLOUD : list_cloud})
  # df_custom["Day"] = list_dayofweek
  df_custom["Date Filter"] = datefilter

  # Filter: Gust & Precip. & Datetime
  df_custom_filtered = df_custom[(df_custom[COL_GUST] <= maxwind_kph)\
                                 & (df_custom[COL_PRECIP] <= maxprecip_mm)\
                                 & (df_custom[COL_CLOUD] <= maxclouds_pcnt)\
                                 & (df_custom["Date Filter"])\
                                 ].drop(columns=["Date Filter", COL_PRECIP]) 
  
  # Verbose
  if is_verbose:
    print("Report generated on: " + datetimenow.strftime("%A %d. %B %Y"))
    print('{}\t{}\t{}'.format(COL_TIME, df_custom_filtered.columns[1], df_custom_filtered.columns[2]))
    for index, row in df_custom_filtered.iterrows():
      for e in row:
        print(e, end="\t\t")
      print('')
    print('\n')

  return df_custom_filtered[COL_TIME]
  

# **Output**
Displays the date & time (hourly) that are suitable for flying.

* Two types of report:
> * detailed_report = {time : weather conditions} for each location
> * summary_report = {time : Yes/No to fly} of all locations


In [4]:
detailed_report = {} 
location_names = []

for key in locations:
  # TODO: parse optional arguments, ie. suffix -> ;67 -> filter out only Saturday and Sunday rows.
  name = locations[key].split(";")

  if not name[0] in location_names:
    location_names.append(name[0])
  
  _flytimes = get_flytimes(key, name[0], MAX_WIND_kph, MAX_PRECIP_mm, MAX_CLOUDS_pcnt, is_verbose=True)
  for str_datetime in _flytimes:
    if not str_datetime in detailed_report:
      detailed_report[str_datetime] = []
    detailed_report[str_datetime].append(name[0])

detailed_report = collections.OrderedDict(sorted(detailed_report.items()))

https://api.open-meteo.com/v1/forecast?latitude=22.234&longitude=114.1593&hourly=precipitation,cloudcover,windgusts_10m&timezone=auto
Weather Forecast: Ap Lei Chau 
GPS: 22.23398579735171, 114.15930196830752
Report generated on: Wednesday 26. October 2022
Datetime (Asia/Hong_Kong)	Gust (km/h)	Cloud Cover (%)
2022-10-27T07:00 Thursday  		30.6		26		
2022-10-27T08:00 Thursday  		29.5		22		
2022-10-27T09:00 Thursday  		34.6		26		
2022-10-27T16:00 Thursday  		34.9		0		
2022-10-27T17:00 Thursday  		32.8		0		
2022-10-28T07:00 Friday    		18.4		16		
2022-10-28T08:00 Friday    		15.5		0		
2022-10-28T09:00 Friday    		20.2		0		
2022-10-28T10:00 Friday    		25.2		0		
2022-10-28T11:00 Friday    		26.6		0		
2022-10-28T12:00 Friday    		28.4		0		
2022-10-28T13:00 Friday    		31.3		10		
2022-10-28T14:00 Friday    		31.3		3		
2022-10-28T15:00 Friday    		28.8		5		
2022-10-28T16:00 Friday    		25.2		0		
2022-10-28T17:00 Friday    		21.2		27		
2022-10-29T07:00 Saturday  		20.2		35		
2022-10-29T08:00 Sat

In [5]:
summary_report = {} 
for str_time in detailed_report:
  for str_name in location_names:
    if not str_name in summary_report:
      summary_report[str_name] = []
    if str_name in detailed_report[str_time]:
      summary_report[str_name].append("✓")
    else:
      summary_report[str_name].append(" ")

df_summary_report = pd.DataFrame(summary_report).assign(time=list(detailed_report.keys()))
cols = df_summary_report.columns.tolist()
cols = cols[-1:] + cols[:-1]
df_summary_report = df_summary_report[cols]

In [6]:
# Formatting

def highlight_max(s):
    is_max = s == s.max() 
    return ['background-color: #FFFFE0' if v else '' for v in is_max]

df_formatted = df_summary_report.style\
  .apply(highlight_max, subset=location_names)\
  .set_table_styles([dict(selector="th",props=[('max-width', '50px')])])\
  .set_properties(**{'text-align': 'left'})\
  .hide_index()


In [7]:
df_formatted

time,Ap Lei Chau,Dragon's Back,Mui Wo,Victoria Park,Peng Chau,Clear Water Bay,Ma On Shan,Tap Mun,Tung Ping Chau
2022-10-26T14:00 Wednesday,,,,,,,,✓,✓
2022-10-26T15:00 Wednesday,,,,,,,,✓,✓
2022-10-26T16:00 Wednesday,,,,,,,,✓,✓
2022-10-26T17:00 Wednesday,,,,,,,,✓,✓
2022-10-27T07:00 Thursday,✓,✓,✓,✓,✓,✓,✓,✓,✓
2022-10-27T08:00 Thursday,✓,,✓,✓,✓,,✓,✓,✓
2022-10-27T09:00 Thursday,✓,,✓,,✓,,,✓,✓
2022-10-27T10:00 Thursday,,,,,,,,✓,✓
2022-10-27T11:00 Thursday,,,,,,,,✓,✓
2022-10-27T12:00 Thursday,,,,,,,,✓,✓
